In [88]:
import os
import streamlit as st
import pickle
from langchain import OpenAI
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

In [89]:
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

True

# User-Interface 

In [90]:
st.title("News research Tool")
st.sidebar.title("News Article URLs")

for i in range(3):
    st.sidebar.text_input(f"URL {i+1}")
    
process=st.sidebar.button("Process URLs")    

if process:
    pass

In [92]:
# %run main.py

UsageError: Line magic function `%streamlit` not found.


# Loaders

In [67]:
from langchain.document_loaders import TextLoader

In [70]:
loader=TextLoader("nvidia.txt")
data=loader.load()

In [71]:
 data[0].metadata

{'source': 'nvidia.txt'}

In [18]:
!pip install unstructured libmagic python-magic python-magic-bin

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/981.5 kB ? eta -:--:--
     ------------- ------------------------ 337.9/981.5 kB 7.0 MB/s eta 0:00:01
     ------------------------------------  972.8/981.5 kB 10.2 MB/s eta 0:00:01
     -------------------------------------- 981.5/981.5 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
INFO: pip is looking at multiple versions of unstructured-client to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of unstructured-client to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolv

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [29]:
from langchain.document_loaders import UnstructuredURLLoader

In [32]:
loader=UnstructuredURLLoader(urls=["https://www.coursera.org/articles/what-is-data-analysis-with-examples",
                           "https://www.coursera.org/in/articles/what-does-a-data-analyst-do-a-career-guide"])

In [33]:
data=loader.load()
len(data)

2

In [37]:
data[0].metadata

{'source': 'https://www.coursera.org/articles/what-is-data-analysis-with-examples'}

# Text Splitting

In [41]:
# Random text from wikipedia

text='''Oppenheimer is a 2023 epic biographical thriller film[a] written, directed, and produced by Christopher Nolan.[8] It follows the life of J. Robert Oppenheimer, the American theoretical physicist who helped develop the first nuclear weapons during World War II. Based on the 2005 biography American Prometheus by Kai Bird and Martin J. Sherwin, the film chronicles Oppenheimer's studies, his direction of the Los Alamos Laboratory, and his fall from grace after his 1954 security hearing. Cillian Murphy stars as Oppenheimer, alongside Robert Downey Jr. as the United States Atomic Energy Commission member Lewis Strauss. The ensemble supporting cast includes Emily Blunt, Matt Damon, Florence Pugh, Josh Hartnett, Casey Affleck, Rami Malek and Kenneth Branagh.

Oppenheimer was announced in September 2021. It is Nolan's first film not distributed by Warner Bros. Pictures since Memento (2000), due to his conflicts regarding the studio's simultaneous theatrical and HBO Max release schedule.[9] Murphy was the first cast member to sign on the following month, with the rest joining between November 2021 and April 2022. Pre-production began by January 2022, and filming took place from February to May. The cinematographer, Hoyte van Hoytema, used a combination of IMAX 65 mm and 65 mm large-format film, including, for the first time, scenes in IMAX black-and-white film photography. As with many of his previous films, Nolan used extensive practical effects, with minimal compositing.

Oppenheimer premiered at Le Grand Rex in Paris on July 11, 2023, and was theatrically released in the US and the UK ten days later by Universal. Its concurrent release with Warner Bros.'s Barbie was the catalyst of the "Barbenheimer" phenomenon, encouraging audiences to see both films as a double feature. Oppenheimer grossed $970 million worldwide, becoming the third-highest-grossing film of 2023, the highest-grossing World War II-related film, the highest-grossing biographical film and the second-highest-grossing R-rated film.

Among its many accolades, Oppenheimer won seven Academy Awards, including Best Picture, Best Director, Best Actor for Murphy and Best Supporting Actor for Downey. It also won five Golden Globe Awards (including Best Motion Picture – Drama) and seven British Academy Film Awards (including Best Film), and was named one of the top ten films of 2023 by the National Board of Review and the American Film Institute.'''

# Why Langchain is used?
'Slice' operator may cut down words, 'for loop' is not suitable for bigger chunks data. on the other hand, Langchain provides an API so manual worl is limited down.

In [43]:
text[:200]

'Oppenheimer is a 2023 epic biographical thriller film[a] written, directed, and produced by Christopher Nolan.[8] It follows the life of J. Robert Oppenheimer, the American theoretical physicist who h'

In [52]:
from langchain.text_splitter import CharacterTextSplitter

splitter=CharacterTextSplitter(
    separator="\n",
    chunk_size=200,
    chunk_overlap=0
)

In [53]:
chunks=splitter.split_text(text)
len(chunks)

Created a chunk of size 760, which is longer than the specified 200
Created a chunk of size 725, which is longer than the specified 200
Created a chunk of size 533, which is longer than the specified 200


4

In [54]:
chunks

["Oppenheimer is a 2023 epic biographical thriller film[a] written, directed, and produced by Christopher Nolan.[8] It follows the life of J. Robert Oppenheimer, the American theoretical physicist who helped develop the first nuclear weapons during World War II. Based on the 2005 biography American Prometheus by Kai Bird and Martin J. Sherwin, the film chronicles Oppenheimer's studies, his direction of the Los Alamos Laboratory, and his fall from grace after his 1954 security hearing. Cillian Murphy stars as Oppenheimer, alongside Robert Downey Jr. as the United States Atomic Energy Commission member Lewis Strauss. The ensemble supporting cast includes Emily Blunt, Matt Damon, Florence Pugh, Josh Hartnett, Casey Affleck, Rami Malek and Kenneth Branagh.",
 "Oppenheimer was announced in September 2021. It is Nolan's first film not distributed by Warner Bros. Pictures since Memento (2000), due to his conflicts regarding the studio's simultaneous theatrical and HBO Max release schedule.[9]

In [55]:
for chunk in chunks:
    print(len(chunk))

760
725
533
412


## To avoid exceeding the chunk size limit

In [59]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

recursive=splitter=RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", " "],
    chunk_size=200,
    chunk_overlap=0
)

chunks=recursive.split_text(text)
len(chunks)

14

In [60]:
for chunk in chunks:
    print(len(chunk))

198
191
196
172
189
192
198
143
198
195
138
199
196
15


# FAISS Index - Facebook AI Similarity Search

In [63]:
!pip install faiss-cpu
!pip install sentence-transformers

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/14.5 MB ? eta -:--:--
    --------------------------------------- 0.3/14.5 MB 9.2 MB/s eta 0:00:02
   - -------------------------------------- 0.5/14.5 MB 10.9 MB/s eta 0:00:02
   ---- ----------------------------------- 1.5/14.5 MB 11.6 MB/s eta 0:00:02
   ---- ----------------------------------- 1.8/14.5 MB 10.1 MB/s eta 0:00:02
   ------ --------------------------------- 2.3/14.5 MB 11.2 MB/s eta 0:00:02
   ------- -------------------------------- 2.9/14.5 MB 10.9 MB/s eta 0:00:02
   ---------- ----------------------------- 3.7/14.5 MB 11.1 MB/s eta 0:00:01
   ------------ --------------------------- 4.6/14.5 MB 12.3 MB/s eta 0:00:01
   ------------- -------------------------- 4.8/14.5 MB 12.8 MB/s eta 0:00:01
   --------------- ------------------------ 5.6/14.5 MB 12.4 MB/s eta 0:00:01
   ----------------- ---------------------- 6.4/14.5 MB 12.9 MB/s eta 0:0

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/171.5 kB ? eta -:--:--
   --------------------------------------- 171.5/171.5 kB 10.7 MB/s eta 0:00:00
